# Computing properties and automatic differentiation

Even though we have now spent about quite some notebooks discussing the SCF, its convergence and errors, the resulting quantity --- the DFT total energy --- is not very interesting.

- From a physical point of view the total energy is arbitrary and can always be shifted around.
- What is of great interest, however, are **differences** or **changes** in the energy
  in response to a perturbation.
- In fact such responses often represent quantities, which are directly measurable
  in experiment (or are at least they are closely linked to measurements).
- A few examples:
  * *dipole moment* (response of the energy to a change in electric field)
  * *forces* (response to a change in atomic positions)
  * ...

Notice that "response" is just a different term for "taking an energy derivative". So that's a good opportunity to try some automatic differentation tools in combination with DFTK.

## Computing forces

Given a crystal with atomic positions $x$ and DFT ground state energy $E_\text{SCF}$ (implicitly depending on $x$ via the external potential $V_\text{ext}$), the force is defined as

$$ \text{Force} = - \frac{d E_\text{SCF}}{d x} $$

For a silicon crystal one could define the DFT ground state in terms of a shift vector ($x$) between
both silicon atoms as follows:

In [ ]:
using DFTK

function compute_silicon(x)
    a = 10.26
    lattice = a / 2 * [[0 1 1.];
                       [1 0 1.];
                       [1 1 0.]]
    Si = ElementPsp(:Si, psp=load_psp("hgh/lda/si-q4"))
    atoms = [Si => [ones(3)/8, -ones(3)/8 + x]]
    
    T = eltype(x)
    model  = model_DFT(Array{T}(lattice), atoms,[:lda_x, :lda_c_vwn]; symmetries=false)
    basis  = PlaneWaveBasis(model; Ecut=13, kgrid=[2, 2, 2]);
    
    self_consistent_field(basis; tol=1e-14, callback=identity)
end

With this function we compute the silicon force in DFTK as:

In [ ]:
ε = 1e-2randn(3)  # Slight distortion to get something non-boring
scfres = compute_silicon(ε)
compute_forces(scfres)[1][2]  # Select force on displaced silicon atom

The `compute_forces` uses analytical derivative expressions implemented manually inside DFTK. Implementing these by hand for a few standard properties and DFT models is fine. However, the number of DFT models is rather large and so is the number of interesting properties. Some properties even require higher order derivatives, e.g. 2nd and 3rd energy derivatives are also not unusual.

Deriving and implementing all these derivatives is tedious, error-prone and time consuming. Let's see if we can at least replicate this force result using some of Julia's **automatic differentiation** tools.

First we baseline with **finite differences**. Instead of running the SCF algorithm twice (once for $x$ and once for $x + \epsilon$), we explicitly exploit the *Hellmann-Feynman theorem*,
which states that near an SCF ground state

$$ \frac{d E_\text{SCF}}{d x} = \frac{\partial E_\text{SCF}}{\partial x} + \frac{\partial E_\text{SCF}}{\partial \Psi} \frac{\partial \Psi}{\partial x} = \frac{\partial E_\text{SCF}}{\partial x}$$

i.e. that the dependency of $E_\text{SCF}$ on the orbitals $\Psi = \{\psi_i\}$ does not need to be considered.

With this simplication:

In [ ]:
using FiniteDiff
scfres = compute_silicon(ε)

function recompute_silicon_energy(x)
    T = eltype(x)
    model = scfres.basis.model
    
    Si = ElementPsp(:Si, psp=load_psp("hgh/lda/si-q4"))
    atoms = [Si => [ones(3)/8, -ones(3)/8 + x]]
    new_model  = model_DFT(Array{T}(model.lattice),
                           atoms,
                           [:lda_x, :lda_c_vwn];
                           symmetries=false)
    new_basis  = PlaneWaveBasis(new_model; Ecut=13, kgrid=[2, 2, 2]);
    
    ρ = DFTK.compute_density(new_basis, scfres.ψ, scfres.occupation)
    energy_hamiltonian(new_basis, scfres.ψ, scfres.occupation; ρ=scfres.ρ).E.total
end

- FiniteDiff.finite_difference_gradient(recompute_silicon_energy, ε)

Sort of agrees ... next we try `ForwardDiff` (tip of the hat to our GSoC Niklas Schmitz, who recently made this possible):

In [ ]:
using ForwardDiff
- ForwardDiff.gradient(recompute_silicon_energy, ε)

## Computing stresses

To close off this section on property computation, let's use **forward-mode AD** to implement a property computation that is not yet available in DFTK.

We will consider the computation of stresses. Denoting the `lattice` matrix by $\textbf{L}$ for simplicty, the stress is computed as

$$ \text{Stress} = \frac{1}{\text{det}(\mathbf{L})} \left. \frac{d E_\text{SCF}}{d [(I + \mathbf{M}) \, \mathbf{L}]} \right|_{\mathbf{M}=0} = \frac{1}{\text{det}(\mathbf{L})} \left. \frac{\partial E_\text{SCF}}{\partial [(I + \mathbf{M}) \, \mathbf{L}]} \right|_{\mathbf{M}=0},
$$

where again we made use of the Hellman-Feynman theorem.

**Exercise 1:** Use the following code fragment to implement stresses using `ForwardDiff`:

In [ ]:
using ForwardDiff
using DFTK
scfres = compute_silicon(zeros(3))

function recompute_silicon_energy_stresses(lattice)
    atoms = scfres.basis.model.atoms
    new_model  = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn]; symmetries=false)
    new_basis  = PlaneWaveBasis(new_model; Ecut=13, kgrid=[2, 2, 2]);
    ρ = DFTK.compute_density(new_basis, scfres.ψ, scfres.occupation)
    energy_hamiltonian(new_basis, scfres.ψ, scfres.occupation; ρ=scfres.ρ).E.total
end

L = scfres.basis.model.lattice

# Your code here

#### Takeaways:
- Practical DFT simulations require not only energies, but also energy derivatives.
- While analytical derivatives are fast, implementing higher order derivatives
  by hand is very, very time-consuming.
- Only a subset of possibilities is usually implemented in standard codes (like DFTK)
- Automatic differentation provides an easy way to go beyond the intrinsic properties available in a code


- In DFTK we are currently working on more complicated setups for automatic differentation (reverse mode, going beyond Hellmann-Feynman).